In [9]:
pip install vecstack

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19877 sha256=648fddf22d79a4a04d4b343c63fd6148abfcdd8238db40262af40de0f041c8d3
  Stored in directory: /voc/work/.cache/pip/wheels/b8/d8/51/3cf39adf22c522b0a91dc2208db4e9de4d2d9d171683596220
Successfully built vecstack
DEPRECATION: beakerx-base 2.0.1 has a non-standard dependency specifier ipywidgets<8pandas,>=7.5.1. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of beakerx-base or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [28]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
names = ['Class', 'Alcohol', 'Malic acid', 'Ash',
         'Alcalinity of ash','Magnesium', 'Total phenols',
         'Flavanoids', 'Nonflavanoid phenols', 
         'Proanthocyanis', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

df = pd.read_csv(link, header=None, names=names)
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanis,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [29]:
#define X and y, split data into train and test
X=df.iloc[:,1:] #all rows and all columns except for column 1 'class'
y = df[['Class']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#Avoids index error with new XGBclassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

/usr/local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
#create the models using KNN, RandomForest, and XGBClassifier
models = [KNeighborsClassifier(n_neighbors=5, n_jobs=-1,), 
          RandomForestClassifier(random_state=0, n_jobs=-1,n_estimators=100, max_depth=5),
          XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, n_estimators=100, max_depth=5)]

In [32]:
#S train and S test, we will be stacking the models by passing train of X and y

S_train, S_test = stacking(models, X_train, y_train, X_test,
                           regression=False, mode='oof_pred_bag', needs_proba=False,
                           save_dir=None, metric=accuracy_score,n_folds=4, stratified=True,
                           shuffle=True, random_state=0, verbose=2)

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.69444444]
    fold  1:  [0.63888889]
    fold  2:  [0.62857143]
    fold  3:  [0.65714286]
    ----
    MEAN:     [0.65476190] + [0.02509117]
    FULL:     [0.65492958]

model  1:     [RandomForestClassifier]
    fold  0:  [0.97222222]
    fold  1:  [0.97222222]
    fold  2:  [1.00000000]
    fold  3:  [1.00000000]
    ----
    MEAN:     [0.98611111] + [0.01388889]
    FULL:     [0.98591549]

model  2:     [XGBClassifier]
    fold  0:  [0.94444444]
    fold  1:  [0.94444444]
    fold  2:  [0.97142857]
    fold  3:  [1.00000000]
    ----
    MEAN:     [0.96507937] + [0.02297479]
    FULL:     [0.96478873]



In [ ]:
#looking at the mean of each model, we can understand the accuracy of each fold